In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import time
import os
import csv
import torch
from validate import validate
from networks.resnet import resnet50
from options.test_options import TestOptions
import networks.resnet as resnet
import numpy as np
import random
import pandas as pd

DetectionTests = {
                'ForenSynths': { 'dataroot'   : '../../datasets/ForenSynths/',
                                 'no_resize'  : True,
                                 'no_crop'    : False,
                               },

           'GANGen-Detection': { 'dataroot'   : '../../datasets/GANGen-Detection/',
                                 'no_resize'  : True,
                                 'no_crop'    : False,
                               },

         'DiffusionForensics': { 'dataroot'   : '../../datasets/DiffusionForensics/',
                                 'no_resize'  : True,
                                 'no_crop'    : False,
                               },

        'UniversalFakeDetect': { 'dataroot'   : '../../datasets/UniversalFakeDetect/',
                                 'no_resize'  : True,
                                 'no_crop'    : False,
                               },

        'Diffusion1kStep': { 'dataroot'   : '../../datasets/Diffusion1kStep/',
                                 'no_resize'  : True,
                                 'no_crop'    : False,
                               },

                 }


opt = TestOptions()

# params
opt.model_path = './checkpoints/test-4class-resnet-car-cat-chair-horse2024_08_09_22_33_48/model_epoch_last.pth'
opt.batch_size = 1024
opt.isTrain = False
opt.mode = 'binary'
opt.class_bal = False
opt.eval = True
opt.num_threads = 8
opt.rz_interp = ['bilinear']
opt.loadSize = 256
opt.blur_prob = 0
opt.jpg_prob = 0
opt.cropSize = 224

print(f'Model_path {opt.model_path}')
torch.cuda.set_device('cuda:0')

# get model
model = resnet50(num_classes=1)
state_dict = torch.load(opt.model_path, map_location='cpu')
model.load_state_dict(state_dict['model'])
model.cuda()
model.eval()

results = []

for testSet in DetectionTests.keys():
    dataroot = DetectionTests[testSet]['dataroot']
    print(testSet)

    accs = []
    aps = []
    print(time.strftime("%Y_%m_%d_%H_%M_%S", time.localtime()))
    for v_id, val in enumerate(os.listdir(dataroot)):
        opt.dataroot = '{}/{}'.format(dataroot, val)
        opt.classes  = '' #os.listdir(opt.dataroot) if multiclass[v_id] else ['']
        opt.no_resize = DetectionTests[testSet]['no_resize']
        opt.no_crop   = DetectionTests[testSet]['no_crop']
        acc, ap, _, _, _, _ = validate(model, opt)
        accs.append(acc)
        aps.append(ap)
        print("({} {:12}) acc: {:.1f}; ap: {:.1f}".format(v_id, val, acc*100, ap*100))
        results.append({'group': testSet, 'id': v_id, 'dataset': val, 'accuracy': acc * 100, 'average_precision': ap * 100})
    results.append({'group': testSet, 'id': v_id + 1, 'dataset': 'Mean', 'accuracy': np.array(accs).mean() * 100, 'average_precision': np.array(aps).mean() * 100})    #TODO: изменить dataframe
    print("({} {:10}) acc: {:.1f}; ap: {:.1f}".format(v_id+1,'Mean', np.array(accs).mean()*100, np.array(aps).mean()*100))
    print('*'*25) 

df = pd.DataFrame(results)
df.to_csv('CNNDetection_MyTrain_results.csv', index=False)


Model_path ./checkpoints/test-4class-resnet-car-cat-chair-horse2024_08_09_22_33_48/model_epoch_last.pth


/tmp/ipykernel_824807/996949208.py:64: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(opt.model_path, map_location='cpu')


ForenSynths
2024_08_10_06_53_50
(0 biggan      ) acc: 65.0; ap: 74.8
(1 crn         ) acc: 59.6; ap: 96.8
(2 cyclegan    ) acc: 67.2; ap: 77.2
(3 deepfake    ) acc: 95.3; ap: 98.3
(4 gaugan      ) acc: 58.4; ap: 61.0
(5 imle        ) acc: 59.6; ap: 91.1
(6 progan      ) acc: 99.6; ap: 100.0
(7 san         ) acc: 73.3; ap: 86.7
(8 seeingdark  ) acc: 92.5; ap: 97.9
(9 stargan     ) acc: 99.9; ap: 100.0
(10 stylegan    ) acc: 81.4; ap: 93.7
(11 stylegan2   ) acc: 75.5; ap: 99.3
(12 whichfaceisreal) acc: 77.5; ap: 92.9
(13 Mean      ) acc: 77.3; ap: 90.0
*************************
GANGen-Detection
2024_08_10_07_04_36
(0 AttGAN      ) acc: 55.8; ap: 71.9
(1 BEGAN       ) acc: 84.2; ap: 99.9
(2 MMDGAN      ) acc: 77.2; ap: 98.4
(3 RelGAN      ) acc: 95.0; ap: 99.6
(4 S3GAN       ) acc: 63.3; ap: 74.6
(5 SNGAN       ) acc: 51.0; ap: 79.9
(6 STGAN       ) acc: 56.0; ap: 98.6
(7 CramerGAN   ) acc: 80.5; ap: 98.8
(8 InfoMaxGAN  ) acc: 51.9; ap: 83.3
(9 Mean      ) acc: 68.3; ap: 89.4
************

In [2]:
torch.cuda.empty_cache()